Connected to .venv (Python 3.10.16)

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna 
from optuna.visualization import plot_optimization_history



from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/dataframeforpypots.parquet")

product_id = "6d48e8c49e3ebecad7aef2f1e069ec53"

## Adding engine types and assign onehot encoder, and merge df  
con = data_insight.setup_duckdb()
con.sql("SET enable_progress_bar=true")
engine_type = con.sql(f"SELECT engine_type FROM shipinfo WHERE productId == '{product_id}'").df().engine_type.item()
df['engine_type']=engine_type

#Apply onehotencoder 
encoder = OneHotEncoder(sparse_output=False)
encoded_engine_type = encoder.fit_transform(df[['engine_type']])
# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_engine_type, columns=encoder.get_feature_names_out(['engine_type']))
# Concatenate the original DataFrame with the encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)
#drop? not sure yet. 
df.drop('engine_type', axis=1,inplace=True)

# filter df with engine running (changed into 10/60 revolutions)
df1 = df[df["fr_eng"] > (10/60)]


# Check nan values in each column
for col in df1.columns:
    print(f"Column {col} has {df1[col].isna().sum()} NaN values")
    missing_rate=df1[col].isna().sum()/len(df1[col])
    print(f"Column {col} has {missing_rate} Missing_rate")

len(df1)

#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
df2=df1[:6000]



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Your data processing code
sensor_cols = [col for col in df2.columns if col != "time"]
data = df2[sensor_cols].values
#¤get feature names for printing mae later 
feature_names=df2[sensor_cols].columns.tolist()
## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")



##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}
data_normalized=data.copy()

data_normalized = np.zeros_like(data_reshaped)  # Initialize the normalized data array
scalers = {}

for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
    data_normalized[:, :, i] = scaler.fit_transform(data_reshaped[:, :, i].reshape(-1, 1)).reshape(data_reshaped.shape[0], data_reshaped.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later



#Split into train, test, val

train_size = int(0.6 * len(data_normalized))
val_size = int(0.2 * len(data_normalized))
test_size = len(data_normalized) - train_size - val_size

train_data = data_normalized[:train_size]
val_data = data_normalized[train_size:train_size + val_size]
#test_data= data_normalized[train_size + val_size:]

#Optional: Artificially mask. Mask 20% of the data 
def mcar_f(X, mask_ratio=0.2):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_data)
val_X_ori=val_data.copy() #Ground truth


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))


#3. Set aside a part of dataset for test_set
df_test=df1[6000:8000]  #Even with 10000 rows of dataset fot testing, after dropping nan value in rows, there is none left 
#2. Drop nan rows 
df_test=df_test.dropna(axis=0,how='all')
print("Size of the test set:", len(df_test))
data_test=df_test[sensor_cols].values




n_features = data_test.shape[1]  # exclude the time column
n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data_test.shape[0] // n_steps 
#Reshape and apply mcar to the test set 
data_test=data_test[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)


data_test_normalized=np.zeros_like(data_test)  # Initialize the normalized data array
for i in range(data_test.shape[2]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
    data_test_normalized[:, :, i] = scaler.fit_transform(data_test[:, :, i].reshape(-1, 1)).reshape(data_test.shape[0], data_test.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

test_X_masked, test_mask =mcar_f(data_test_normalized)
test_X_ori=data_test_normalized.copy() #Ground truth


class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")


# initialize the model (from example)
saits = SAITS(
    n_steps=data_normalized.shape[1],
    n_features=data_normalized.shape[2],
    n_layers=2, #deep network(4) for long sequences, here is revised for 2 since it always shows cuda error 
    d_model=512,  #d_modle must equal n_heads * d_k
    d_ffn=512,
    n_heads=8,
    d_k=64,
    d_v=64,
    dropout=0.1,
    attn_dropout=0.1,
    diagonal_attention_mask=True,  # otherwise the original self-attention mechanism will be applied
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=4,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=5,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=4, #initially was 3, tried to increase to see more possibilities 
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=device, 
    # set the path for saving tensorboard and trained model files 
    saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.

    model_saving_strategy="best",
)





# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
#use original missigness for trainig, use the mcar masked points as ground truth
saits.fit(train_set={"X": train_data}, val_set={"X": val_X_masked , "X_ori":val_X_ori })


##drop null values in test set 

# Check for NaN values across the entire array
#nan_mask = np.isnan(test_X).any(axis=(1, 2))

# Filter out samples (rows) that contain NaN values
#test_X_clean = test_X[~nan_mask]
#test_X_ori = test_X_clean
#test_X_masked, test_mask = mcar_f(test_X_clean)

# the testing stage, impute the originally-missing values and artificially-missing values in the test set
# Convert the numpy array to a dictionary
test_set_dict = {"X": test_X_masked}
test_imputation = saits.predict(test_set_dict)

#change back to array for applying inverse scale later
test_imputation_array = test_imputation["imputation"]


#clear cuda after prediction if on cuda

# Ensure saits_imputation and test_X are numpy arrays
#if not isinstance(saits_imputation, np.ndarray):
    #saits_imputation = np.array(saits_imputation)
#if not isinstance(test_X, np.ndarray):
     #test_X = np.array(test_X)

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  # Move the return statement outside the loop

    
#Apply function to the dataset 
test_imputation_denorm=inverse_scale(test_imputation_array,scalers)
test_X_ori_denorm=inverse_scale(test_X_ori,scalers)

if args.cuda:
    torch.cuda.empty_cache() 


#Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature=[]
for i in range(n_features):
    #Extract imputation and ground truth for feature i
    imputation_i=test_imputation_denorm[:,:,i]
    ground_truth_i=test_X_ori_denorm[:,:,i]
    mask_i=test_mask[:,:,i]
    # Check for NaN values
    if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
        print(f"NaN values detected in feature {i}")
        continue  # Skip this feature if NaN values are found
    #Filter only artificially masked positions
    mae_i=calc_mae(imputation_i,ground_truth_i,mask_i)
    mae_per_feature.append(mae_i)
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")



# calculate average MAE 
avg_mae=np.mean(mae_per_feature)
print(f"Testing mean absolute error: {avg_mae:.4f}")


##Current error is on the cuda 
#test memory
#def test_memory(in_size=100, out_size=10,hidden_size=100,optimizer_type=torch.optim.Adam, batch_size=1, device=0):
   # sample_input=torch.randn(batch_size,in_size)
  #  model=saits,
   # optimizer=optimizer_type(model.parameters(),lr=1e-3)
   # print("Beginning mem:",torch.cuda.memory_allocated(device))
   # model.to(device)
   # print()
   # for i in range(3):
   #     print("Iteration",i)
   #     a=torch.cuda.memory_allocated(device)
   #     out=model(sample_input.to(device)).sum()
   #     b=torch.cuda.memory_allocated(device)
   #     print("2-After forward pass", torch.cuda.memory_allocated(device))
   #     print("2-Memory consumed by forward pass", b-a)
   #     out.backward()
   #     print("3-After backward pass", torch.cuda.memory_allocated(device))
   #     optimizer.step()
   #     print("4-After optimizer step", torch.cuda.memory_allocated(device))

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 

>>> con.sql("SHOW TABLES;")
┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ shipinfo   │
│ site_infos │
│ timeseries │
└────────────┘


>>> con.sql("DESCRIBE shipinfo;")
┌──────────────────────┬───────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │                      colu

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Size of the test set: 2000


/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Using CPU


2025-05-13 11:06:07 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 6,702,664
2025-05-13 11:06:19 [INFO]: Epoch 001 - training loss (MAE): 0.6550, validation MSE: 0.0642
2025-05-13 11:06:25 [INFO]: Epoch 002 - training loss (MAE): 0.4644, validation MSE: 0.0406
2025-05-13 11:06:30 [INFO]: Epoch 003 - training loss (MAE): 0.3417, validation MSE: 0.3221
2025-05-13 11:06:35 [INFO]: Epoch 004 - training loss (MAE): 0.2634, validation MSE: 0.3120
2025-05-13 11:06:40 [INFO]: Epoch 005 - training loss (MAE): 0.2140, validation MSE: 0.5687
2025-05-13 11:06:40 [INFO]: Finished training. The best model is from epoch#2.
2025-05-13 11:06:40 [INFO]: Saved the model to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250513_T110607/SAITS.pypots


MAE for fr_eng: 0.0293
MAE for te_exh_cyl_out__0: 5.5161
MAE for te_exh_cyl_out__1: 4.8000
MAE for te_exh_cyl_out__2: 3.2818
MAE for te_exh_cyl_out__3: 3.2495
MAE for te_exh_cyl_out__4: 6.3969
MAE for te_exh_cyl_out__5: 2.6999
MAE for te_exh_cyl_out__6: 4.4496
MAE for pd_air_ic__0: 67.1006
NaN values detected in feature 9
NaN values detected in feature 10
NaN values detected in feature 11
NaN values detected in feature 12
MAE for te_air_ic_out__0: 0.3543
MAE for te_air_ic_out__1: 0.0690
NaN values detected in feature 15
NaN values detected in feature 16
MAE for te_seawater: 0.1348
NaN values detected in feature 18
NaN values detected in feature 19
NaN values detected in feature 20
NaN values detected in feature 21
NaN values detected in feature 22
NaN values detected in feature 23
NaN values detected in feature 24
NaN values detected in feature 25
MAE for pr_baro: 318.5794
NaN values detected in feature 27
NaN values detected in feature 28
NaN values detected in feature 29
NaN values d

In [ ]:
df_test["pr_cyl_comp__7"].std()

369200.2200321264

In [ ]:
df_test["pr_air_scav"].std()

9568.48555028183

In [ ]:
df_test["re_eng_load"].std()

0.021682595958135226

In [ ]:
df_test["te_air_scav_rec"].std()


0.23457730763365267

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna 
from optuna.visualization import plot_optimization_history



from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/dataframeforpypots.parquet")

product_id = "6d48e8c49e3ebecad7aef2f1e069ec53"

## Adding engine types and assign onehot encoder, and merge df  
con = data_insight.setup_duckdb()
con.sql("SET enable_progress_bar=true")
engine_type = con.sql(f"SELECT engine_type FROM shipinfo WHERE productId == '{product_id}'").df().engine_type.item()
df['engine_type']=engine_type

#Apply onehotencoder 
encoder = OneHotEncoder(sparse_output=False)
encoded_engine_type = encoder.fit_transform(df[['engine_type']])
# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_engine_type, columns=encoder.get_feature_names_out(['engine_type']))
# Concatenate the original DataFrame with the encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)
#drop? not sure yet. 
df.drop('engine_type', axis=1,inplace=True)

# filter df with engine running (changed into 10/60 revolutions)
df1 = df[df["fr_eng"] > (10/60)]


# Check nan values in each column
for col in df1.columns:
    print(f"Column {col} has {df1[col].isna().sum()} NaN values")
    missing_rate=df1[col].isna().sum()/len(df1[col])
    print(f"Column {col} has {missing_rate} Missing_rate")

len(df1)

#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
df2=df1[:10000]



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Your data processing code
sensor_cols = [col for col in df2.columns if col != "time"]
data = df2[sensor_cols].values
#¤get feature names for printing mae later 
feature_names=df2[sensor_cols].columns.tolist()
## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")



##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}
data_normalized=data.copy()

data_normalized = np.zeros_like(data_reshaped)  # Initialize the normalized data array
scalers = {}

for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
    data_normalized[:, :, i] = scaler.fit_transform(data_reshaped[:, :, i].reshape(-1, 1)).reshape(data_reshaped.shape[0], data_reshaped.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later



#Split into train, test, val

train_size = int(0.6 * len(data_normalized))
val_size = int(0.2 * len(data_normalized))
test_size = len(data_normalized) - train_size - val_size

train_data = data_normalized[:train_size]
val_data = data_normalized[train_size:train_size + val_size]
test_data= data_normalized[train_size + val_size:]

#Optional: Artificially mask. Mask 20% of the data 
def mcar_f(X, mask_ratio=0.2):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_data)
val_X_ori=val_data.copy() #Ground truth

test_X_masked, test_mask =mcar_f(test_data)
test_X_ori=test_data.copy() #ground truth

#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))


#3. Set aside a part of dataset for test_set
#df_test=df1[6000:8000]  #Even with 10000 rows of dataset fot testing, after dropping nan value in rows, there is none left 
#2. Drop nan rows 
#f_test=df_test.dropna(axis=0,how='all')
#print("Size of the test set:", len(df_test)) 
#data_test=df_test[sensor_cols].values



#n_features = data_test.shape[1]  # exclude the time column
#n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
#n_samples = data_test.shape[0] // n_steps 
#Reshape and apply mcar to the test set 
# #data_test=data_test[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)

#data_test_normalized=np.zeros_like(data_test)  # Initialize the normalized data array
#for i in range(data_test.shape[2]):
  # scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
  # data_test_normalized[:, :, i] = scaler.fit_transform(data_test[:, :, i].reshape(-1, 1)).reshape(data_test.shape[0], data_test.shape[1])
  # scalers[i] = scaler  # Save scalers to inverse-transform later




class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")


# initialize the model (from example)
saits = SAITS(
    n_steps=data_normalized.shape[1],
    n_features=data_normalized.shape[2],
    n_layers=2, #deep network(4) for long sequences, here is revised for 2 since it always shows cuda error 
    d_model=512,  #d_modle must equal n_heads * d_k
    d_ffn=512,
    n_heads=8,
    d_k=64,
    d_v=64,
    dropout=0.1,
    attn_dropout=0.1,
    diagonal_attention_mask=True,  # otherwise the original self-attention mechanism will be applied
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=4,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=5,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=4, #initially was 3, tried to increase to see more possibilities 
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=device, 
    # set the path for saving tensorboard and trained model files 
    saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.

    model_saving_strategy="best",
)





# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
#use original missigness for trainig, use the mcar masked points as ground truth
saits.fit(train_set={"X": train_data}, val_set={"X": val_X_masked , "X_ori":val_X_ori })


##drop null values in test set 

# Check for NaN values across the entire array
#nan_mask = np.isnan(test_X).any(axis=(1, 2))

# Filter out samples (rows) that contain NaN values
#test_X_clean = test_X[~nan_mask]
#test_X_ori = test_X_clean
#test_X_masked, test_mask = mcar_f(test_X_clean)

# the testing stage, impute the originally-missing values and artificially-missing values in the test set
# Convert the numpy array to a dictionary
test_set_dict = {"X": test_X_masked}
test_imputation = saits.predict(test_set_dict)

#change back to array for applying inverse scale later
test_imputation_array = test_imputation["imputation"]


#clear cuda after prediction if on cuda

# Ensure saits_imputation and test_X are numpy arrays
#if not isinstance(saits_imputation, np.ndarray):
    #saits_imputation = np.array(saits_imputation)
#if not isinstance(test_X, np.ndarray):
     #test_X = np.array(test_X)

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  # Move the return statement outside the loop

    
#Apply function to the dataset 
test_imputation_denorm=inverse_scale(test_imputation_array,scalers)
test_X_ori_denorm=inverse_scale(test_X_ori,scalers)

if args.cuda:
    torch.cuda.empty_cache() 


#Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature=[]
for i in range(n_features):
    #Extract imputation and ground truth for feature i
    imputation_i=test_imputation_denorm[:,:,i]
    ground_truth_i=test_X_ori_denorm[:,:,i]
    mask_i=test_mask[:,:,i]
    # Check for NaN values
    if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
        print(f"NaN values detected in feature {i}")
        continue  # Skip this feature if NaN values are found
    #Filter only artificially masked positions
    mae_i=calc_mae(imputation_i,ground_truth_i,mask_i)
    mae_per_feature.append(mae_i)
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")



# calculate average MAE 
avg_mae=np.mean(mae_per_feature)
print(f"Testing mean absolute error: {avg_mae:.4f}")


##Current error is on the cuda 
#test memory
#def test_memory(in_size=100, out_size=10,hidden_size=100,optimizer_type=torch.optim.Adam, batch_size=1, device=0):
   # sample_input=torch.randn(batch_size,in_size)
  #  model=saits,
   # optimizer=optimizer_type(model.parameters(),lr=1e-3)
   # print("Beginning mem:",torch.cuda.memory_allocated(device))
   # model.to(device)
   # print()
   # for i in range(3):
   #     print("Iteration",i)
   #     a=torch.cuda.memory_allocated(device)
   #     out=model(sample_input.to(device)).sum()
   #     b=torch.cuda.memory_allocated(device)
   #     print("2-After forward pass", torch.cuda.memory_allocated(device))
   #     print("2-Memory consumed by forward pass", b-a)
   #     out.backward()
   #     print("3-After backward pass", torch.cuda.memory_allocated(device))
   #     optimizer.step()
   #     print("4-After optimizer step", torch.cuda.memory_allocated(device))

>>> con.sql("SHOW TABLES;")
┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ shipinfo   │
│ site_infos │
│ timeseries │
└────────────┘


>>> con.sql("DESCRIBE shipinfo;")
┌──────────────────────┬───────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │                      column_type                      │  null   │   key   │ default │  extra  │
│       varchar        │                        varchar                        │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼───────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ information_validity │ VARCHAR                                               │ YES     │ NULL    │ NULL    │ NULL    │
│ imo_no               │ VARCHAR                                               │ YES     │ NULL    │ NULL    │ NULL    │
│ ship_name            │ VARCHAR                                               │ YES     │

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Using CPU


2025-05-13 11:18:32 [INFO]: Epoch 001 - training loss (MAE): 0.5071, validation MSE: 0.0142
2025-05-13 11:18:38 [INFO]: Epoch 002 - training loss (MAE): 0.3329, validation MSE: 0.2380
2025-05-13 11:18:44 [INFO]: Epoch 003 - training loss (MAE): 0.2246, validation MSE: 0.1346
2025-05-13 11:18:50 [INFO]: Epoch 004 - training loss (MAE): 0.2180, validation MSE: 0.2112
2025-05-13 11:18:57 [INFO]: Epoch 005 - training loss (MAE): 0.2200, validation MSE: 0.3105
2025-05-13 11:18:57 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-05-13 11:18:57 [INFO]: Finished training. The best model is from epoch#1.
2025-05-13 11:18:57 [INFO]: Saved the model to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250513_T111825/SAITS.pypots


MAE for fr_eng: 0.0984
MAE for te_exh_cyl_out__0: 9.4206
MAE for te_exh_cyl_out__1: 7.3118
MAE for te_exh_cyl_out__2: 8.4779
MAE for te_exh_cyl_out__3: 20.4307
MAE for te_exh_cyl_out__4: 17.5698
MAE for te_exh_cyl_out__5: 2.6724
MAE for te_exh_cyl_out__6: 4.7136
MAE for pd_air_ic__0: 69.3274
NaN values detected in feature 9
NaN values detected in feature 10
NaN values detected in feature 11
NaN values detected in feature 12
MAE for te_air_ic_out__0: 2.1295
MAE for te_air_ic_out__1: 0.0704
NaN values detected in feature 15
NaN values detected in feature 16
MAE for te_seawater: 0.0162
NaN values detected in feature 18
NaN values detected in feature 19
NaN values detected in feature 20
NaN values detected in feature 21
NaN values detected in feature 22
NaN values detected in feature 23
NaN values detected in feature 24
NaN values detected in feature 25
MAE for pr_baro: 872.4188
NaN values detected in feature 27
NaN values detected in feature 28
NaN values detected in feature 29
NaN values

In [ ]:
df2["te_air_scav_rec"].std()

2.361538121253254

In [ ]:
df2["re_eng_load"].std()

0.1519045663187785

In [ ]:
df2["in_engine_running_mode"].std()

0.2692834219853418

In [ ]:
df2["pr_baro"].std()

1045.4026959405012

In [ ]:
df2["se_mip__0"].std()

271464.50553187606

In [ ]:
print(imputation_i)

[[1.         0.96806276 0.9697397  ... 0.9695579  1.         1.        ]
 [1.         1.         0.9682254  ... 1.         1.         1.        ]
 [1.         1.         1.         ... 0.97240907 1.         1.        ]
 ...
 [0.9678866  1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         0.96787304 0.97180384 ... 1.         1.         1.        ]]


In [ ]:
print(imputation_i[2])

[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.9565633
 0.9585411  1.         1.         1.         1.         0.9687108
 1.         1.         1.         1.         0.9681137  1.
 0.9667162  1.         1.         1.         1.         1.
 1.         1.         0.9622312  1.         1.         0.9678425
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.9693707  1.         1.         1.         1.
 1.         1.         1.         0.97240907 1.         1.        ]


In [ ]:
print(data[1])

[7.32877785e-01 5.10150000e+02 5.08150000e+02 5.16150000e+02
 5.14150000e+02 5.06150000e+02 5.13150000e+02 5.13150000e+02
 4.20000000e+02            nan            nan            nan
            nan 3.08950000e+02 2.73150000e+02            nan
            nan 2.73150000e+02            nan            nan
            nan            nan            nan            nan
            nan            nan 1.01400000e+05            nan
            nan            nan            nan            nan
            nan            nan            nan 1.60000000e+04
 2.22412109e+04 2.22412109e+04 7.33310046e-01 3.08950000e+02
 2.73150000e+02            nan            nan 4.20000000e+02
 0.00000000e+00            nan            nan 9.27442093e+06
 9.33718567e+06 9.32157135e+06 9.29906693e+06 9.31256409e+06
 9.40771027e+06 9.36192245e+06 9.29957275e+06            nan
            nan            nan            nan 5.59352112e+05
 5.61633301e+05 5.58950806e+05 5.59037781e+05 5.60848999e+05
 5.59201050e+05 7.328777

In [ ]:
print(ground_truth_i)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
